In [1]:
import pandas as pd
pd.set_option('display.max_rows',500)
import numpy as np
import os
import gzip
import re

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

In [3]:
def preparing_data(path, platform):
    df = pd.read_csv(path)
    df['platform'] = platform
    return df

In [62]:
def get_nulls(df):
    return df[df['imdb_rating'].isnull()].shape
    

In [5]:
netflix = preparing_data('data\\kaggle_data\\netflix_titles.csv', 'netflix')

In [6]:
amazon = preparing_data('data\\kaggle_data\\amazon_prime_titles.csv', 'amazon')

In [7]:
disney = preparing_data('data\\kaggle_data\\disney_plus_titles.csv', 'disney')

In [8]:
data = pd.concat([netflix, amazon, disney])

In [3]:
amazon = pd.read_csv('data\\notebook_df\\amazon.csv')


### Selenium extraction

In [4]:
def get_rating(value):
    
    try:
    
        os.environ['WDM_LOG_LEVEL'] = '0'
        browser = webdriver.Chrome(ChromeDriverManager().install())
        wait = WebDriverWait(browser, 10)
        browser.maximize_window()
        browser.get("https://www.imdb.com/")
        browser.implicitly_wait(10)

        search = wait.until(ec.element_to_be_clickable((By.XPATH, '//*[@id="suggestion-search"]')))
        search.send_keys(value)
        browser.implicitly_wait(7)

        search_button = browser.find_element_by_id('suggestion-search-button')
        search_button.click()

        #data_year = str(l.release_year[l.title == value].item())
        data_year = str(data.release_year[data.title == value].to_list()[0])

        try:

            results_table = browser.find_element_by_class_name('findList')
            rows = results_table.find_elements(By.TAG_NAME, "tr")
            col = [row.find_elements(By.TAG_NAME, "td")[1].text for row in rows]
            for rg in range(0, len(col)):
                try:
                    imdb_year = re.findall('\(\d{4}\)',col[rg])[0].replace('(','').replace(')','')
                    if imdb_year == data_year:
                        search_result = wait.until(ec.element_to_be_clickable((By.XPATH, f'/html/body/div[3]/div/div[2]/div[3]/div[1]/div/div[2]/table/tbody/tr[{rg+1}]/td[2]/a')))
                        search_result.click()
                        rating = browser.find_element_by_class_name('AggregateRatingButton__RatingScore-sc-1ll29m0-1.iTLWoV').text
                        browser.close()
                        return rating
                    else:
                        pass

                except: 
                    pass
        except:
            pass
    except:
        pass

In [7]:
amazon_null = amazon[amazon.imdb_rating.isnull()]

In [11]:
pd.set_option('display.max_rows',5000)

In [22]:
amazon_dir = amazon_null[~amazon_null.director.isnull()]

In [23]:
amazon_dir.shape

(3398, 14)

In [5]:
def rating_2(value):
    
    try:
        os.environ['WDM_LOG_LEVEL'] = '0'
        browser = webdriver.Chrome(ChromeDriverManager().install())
        wait = WebDriverWait(browser, 10)
        browser.maximize_window()
        browser.get("https://www.imdb.com/")
        browser.implicitly_wait(10)

        search = wait.until(ec.element_to_be_clickable((By.XPATH, '//*[@id="suggestion-search"]')))
        search.send_keys(value)
        browser.implicitly_wait(7)

        search_button = browser.find_element_by_id('suggestion-search-button')
        search_button.click()
        search_result = wait.until(ec.element_to_be_clickable((By.XPATH, f'/html/body/div[3]/div/div[2]/div[3]/div[1]/div/div[2]/table/tbody/tr[1]/td[2]/a')))
        search_result.click()
        rating = browser.find_element_by_class_name('AggregateRatingButton__RatingScore-sc-1ll29m0-1.iTLWoV').text
        browser.close()
        return rating
    except:
        pass

In [16]:
amazon_null.to_csv(r'C:\Users\juanp\proyectos_varios\streaming_platforms\data\notebook_df\amazon_null.csv' , index = False)